## 1 Project Overview and Setup: Understanding the Purpose

In this project, I'm an analyst at Megaline, a telecom operator. We offer two prepaid plans, Surf and Ultimate. The commercial department has tasked me with figuring out which plan generates more revenue so they can adjust the advertising budget accordingly.

For this preliminary analysis, I'll be looking at a subset of our clients. I have data on 500 Megaline clients, including their demographics, plan usage, and communication activity in 2018. My job is to analyze this data and identify which prepaid plan contributes more to our revenue.

Note: Megaline rounds seconds up to minutes, and megabytes to gigabytes. For calls, each individual call is rounded up: even if the call lasted just one second, it will be counted as one minute. For web traffic, individual web sessions are not rounded up. Instead, the total for the month is rounded up. If someone uses 1025 megabytes this month, they will be charged for 2 gigabytes.

## 2 Initialization

First, import all of the necessary Python libraries for the project.